In [4]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [47]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-46165?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Senior Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, 

In [48]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Senior Software Engineer, ITC",
  "experience": "5-8 years",
  "skills": [
    "Java",
    "Scala",
    "Node.js",
    "TyperScript",
    "Serverless AWS",
    "ReactJs",
    "Vue",
    "Amazon EC2",
    "GCE",
    "Azure",
    "MySQL",
    "DynamoDB",
    "Elastic Search",
    "Jenkins",
    "GitHub",
    "SonarQube",
    "Harness",
    "Prisma",
    "Kafka",
    "New Relic",
    "SignalFx",
    "Splunk"
  ],
  "description": "As a Senior Software Engineer, you will be involved in designing and building secure, highly scalable and innovative services in a continuous delivery model on the Nike cloud platform. This role will partner with leads and architects to deliver web services that are robust, architecturally sound, easily maintainable and highly scalable for various Nike partner experiences and platform."
}
```


In [57]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

In [58]:
type(json_res)

dict

In [10]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [60]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [66]:
links = collection.query(query_texts=job['skills'],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],

In [65]:
job['skills']

['Java',
 'Scala',
 'Node.js',
 'TyperScript',
 'Serverless AWS',
 'ReactJs',
 'Vue',
 'Amazon EC2',
 'GCE',
 'Azure',
 'MySQL',
 'DynamoDB',
 'Elastic Search',
 'Jenkins',
 'GitHub',
 'SonarQube',
 'Harness',
 'Prisma',
 'Kafka',
 'New Relic',
 'SignalFx',
 'Splunk']

In [64]:
job = json_res

type(job)

dict

In [68]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Akshay, a business development executive at InnovateXperts.InnovateXperts is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase InnovateXperts's portfolio: {link_list}
        Remember you are Akshay, BDE at InnovateXperts. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Solutions for Nike's Cloud Platform

Dear Hiring Manager,

I came across the job description for a Senior Software Engineer, ITC at Nike, and I'm excited to introduce InnovateXperts, a leading AI & Software Consulting company that can help fulfill your requirements. With our expertise in designing and building secure, highly scalable, and innovative services, we're confident that we can make a significant impact on your cloud platform.

Our team has extensive experience in working with various technologies, including Java, Scala, Node.js, TypeScript, Serverless AWS, ReactJs, Vue, Amazon EC2, GCE, Azure, MySQL, DynamoDB, Elastic Search, Jenkins, GitHub, SonarQube, Harness, Prisma, Kafka, New Relic, SignalFx, and Splunk. We've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

I'd like to highlight some of our relevant portfolio links that demonstrate our capabil